# generating filters and visibility widgets for tables

we can infer a lot about the expectations of complete interface for interacting with the contents from the inherit type information in a dataframe. we demonstration the steady state html that would for the basis for an accessible experience.

[demo](#gist)

In [1]:
%%
```css
a[href="#gist"] {
    display: inline-block;
    height: 200px;
    width: 100%;
    background: -moz-element(#gist-region);
    background-size: contain;
}
```

'a[href="#gist"] {\n    display: inline-block;\n    height: 200px;\n    width: 100%;\n    background: -moz-element(#gist-region);\n    background-size: contain;\n}\n'

In [2]:
%%
    gists = (await (
        "https://api.github.com/users/tonyfast/gists?page=" + pandas.RangeIndex(1, 5).astype(str)
    ).http.get()).explode().series().set_index("id")
    gists.index.name = "gist_id"
we're going to quickly make a table of my gist from github.

    files = gists.join(gists.files.apply(dict.values).apply(list).explode().series())
unravel all the gists in each gist payload.

    time = files.columns[files.columns.str.endswith("_at")].tolist()
    files[time] = files[time].apply(pandas.to_datetime, axis=1)
    files = files.assign(timespan=files.updated_at - files.created_at)

convert times to their proper types and substract them so out dataframe 
contains integers, datetimes, and timedeltas, types with numerical representations.
    
    files[cat] = files[cat := "language type".split()].astype("category")
    df = files[time + "timespan filename description language type size".split()]
    
now are going to add a few lines of `css` that uses the `--ratio` variables in each numerical cell
to produce visual representation of the cell value in the larger distribution.
it is really interesting how we start to create the cell border form when we do this.
at a glance we can visually observe relative positions of values in a column.
further, scrolling creates really interesting movement that provides user controlled animation.
    
    notebooks = (
        await files[files.language.eq("Jupyter Notebook")].raw_url.http.get()
    ).apply(json.loads).series()
read the `notebooks` and separate out the cells, outputs, and display objects.

    cells = notebooks.cells.enumerate("cell").series()
    outputs = cells.outputs[cells.outputs.fillna("").astype(bool)].enumerate("display").series()
    display_data = outputs['data'].dropna().series()

    df = (df.join(cells.groupby("gist_id").id.count().rename("cells"))
          .join(outputs.groupby("gist_id")["data"].count().rename("outputs")))
    from nbconvert_a11y.table import new; import nbconvert_a11y
    table = df[
        "filename description language type created_at updated_at timespan size cells outputs".split()
    ].sort_values("created_at", ascending=False).table(id="gist").style(
```css
dialog > details >  form > button[formmethod=dialog] {
    display: none;
}
dialog[open] > details > form {
    & > button[formmethod=dialog] {display: unset;}
    & > button[formmethod=dialog] + button {display: none;}
}
```
    )

In [3]:
%%    
    table.style(
```css
dialog > form > button[formmethod=dialog] {
    display: none;
}
dialog[open] > form {
    & > button[formmethod=dialog] {display: unset;}
    & > button[formmethod=dialog] + button {display: none;}
}
td {
    --marker-width: .2ch;
    background: linear-gradient(90deg, 
                                rgba(255, 0, 0, 0) 0% calc(var(--ratio) * 100% - var(--marker-width)),
                                rgba(255, 0, 0, 1) calc(var(--ratio) * 100% - var(--marker-width)) calc(var(--ratio) * 100% + .5 * var(--marker-width)),
                                rgba(255, 0, 0, 0) calc(var(--ratio) * 100% + var(--marker-width)) 100%
                               );
    background-size: 100% 50%;
    background-position-y: 25%;
    background-repeat: no-repeat;
    &:empty::before {
        content: "nan";
    }
}
&, table, tr 
    background: var(--bg, white);
    color: var(--fg, black);
}
```
    )
    table.container.form.append(
        table_filters := new(
            "details", new("summary", "filters"), open=""
        )
    )
    table.container.form.append(
        table_visibility := new(
            "details", new("summary", "visibility"), open=""
        )
    )

* categorical and numerical filters
* time filters

serialize css, attributes and html different. fuckkkk

In [4]:
%%
## creating the column filters

any numeric column, integers, floats, time can be used as a filter for the table.
we build those for


        bounds = []
1. separate the numeric values and make them `input[type=number]`

        numeric = table.object.dtypes.apply(nbconvert_a11y.outputs.is_number)
        stats = table.stats[table.stats.columns[numeric]].T.set_index(table.object.columns[numeric])
        ranges = stats.bs4.input(type="number")
        ranges.bs4.attrs(stats).pipe(bounds.append)

1. separate the timedeltas values and make them `input[type=number]`. preprocessing is different than numbers

        # missing units
        timedeltas = table.object.dtypes.apply(isinstance, args=(numpy.dtypes.TimeDelta64DType,))
        stats = table.stats[table.stats.columns[timedeltas]].T.set_index(table.object.columns[timedeltas]).map(pandas.Timedelta.total_seconds)
        ranges = stats.bs4.input(type="number")
        ranges.bs4.attrs(stats).pipe(bounds.append)

1. separate the times and make them `input[type=datetime-local]`

    
        times = table.object.dtypes.apply(isinstance, args=(pandas.core.dtypes.dtypes.DatetimeTZDtype,))
        stats = table.stats[table.stats.columns[times]].T.set_index(table.object.columns[times]).bs4.to_attribute()
        ranges = stats.bs4.input(type="datetime-local")
        ranges.bs4.attrs(stats).pipe(bounds.append)

1. rejoin them back with the original container

        table_filters.append(
            (ranges := pandas.concat(bounds).reindex(table.object.columns).rename_axis(index="column").dropna().table()).table
        )

In [5]:
%%
## create visibility toggles

1. make an dataframe of `input[type=radio]` from an enum and the current columns in the table we are building.

        visibility = DataFrame(
            None,
            Index(
                Series(nbconvert_a11y.options.Presentation.__members__.keys()).bs4.th(), name="visibility"
            ),
            Series(table.table.thead.tr.select("th")).bs4.pop("aria-sort").apply(copy.copy)
        ).bs4.input(type="radio")

1. `aria-label` for now. need to add ids to the column headers to make `aria-labelledby` work

        visibility.bs4.attrs({
            "aria-label": 
            visibility.index.to_series().apply(getattr, args=("string",)).values[:, None] + " " + 
                  visibility.columns.to_series().apply(getattr, args=("string",)).values[None, :]
        
        })


1. give the same name to radio button row so we get arrow key support

        for i, (k, v) in enumerate(visibility.items()):
            v.bs4.attrs(name=F"visibility-{i}")
1. check the first ones to initialize

        visibility.iloc[0, :].bs4.attrs(checked="")
1.  the columns as the row index makes the preferred keyboard pattern for navigation with native html this is a better reference.

        visibility = visibility.T.rename_axis(index="column").table()


        

1. rejoin the visibility grid with the original container

        table_visibility.append(visibility.table)

In [6]:
    table.container.dialog.details.attrs.update(open="")
    table

gist_id,filename,description,language,type,created_at,updated_at,timespan,size,cells,outputs
90c41d4994f75c594db804aeba56fc26,first_and_second_laws_of_thermodynamics.ipynb,a notebook with revised literate pidgy cells for the first part of thermodyamics course,Jupyter Notebook,text/plain,2024-01-10T21:04:06+00:00,2024-01-10T21:04:39+00:00,33.0 s,20469,16.0,7.0
aa3b16c5a284150e3d727a843b6cefec,axe_types.py,,Python,application/x-python,2023-12-03T21:31:06+00:00,2023-12-03T21:31:06+00:00,0.0 s,8442,,
713ae6c57602c0f85d011421b20d5ea0,BinaryExamples.ipynb,a revision of a pycalphad documentation page as literate program.,Jupyter Notebook,text/plain,2023-11-02T20:59:23+00:00,2023-11-02T21:05:32+00:00,369.0 s,352154,0.0,6.0
c004044b4fe641735031ecf2069cf595,aom.json,an exported accessibility tree from https://iota-school.github.io/notebooks-for-all/branch/extent/exports/html/lorenz-executed-smol.html#14,JSON,application/json,2023-06-06T18:09:18+00:00,2023-06-06T18:09:19+00:00,1.0 s,52330,,
e17946facd998a931527467d646cc822,README.md,extensible notebook schemas,Markdown,text/markdown,2023-02-21T22:32:31+00:00,2023-02-21T22:35:44+00:00,193.0 s,3044,,
99303420edc6021bbeeaa96c522469a1,ummm.py,,Python,application/x-python,2022-12-23T06:16:30+00:00,2022-12-23T06:16:30+00:00,0.0 s,266,,
c8e19ffb2b85f5bf6f5a4b6e76c327cd,abc.py,abc of python modules,Python,application/x-python,2022-12-20T03:08:25+00:00,2022-12-20T03:08:25+00:00,0.0 s,469,,
4e33098a9f3a50805ad99d08e96825a6,graph shit.py,,Python,application/x-python,2022-12-18T21:07:29+00:00,2022-12-18T21:07:29+00:00,0.0 s,4213,,
243d7758d659c406f6e12cbd7b045384,hide_empty_outputs.html,a selection to hide empty jupyterlab output cells,HTML,text/html,2022-12-17T23:08:41+00:00,2022-12-17T23:08:41+00:00,0.0 s,90,,
42612252e0ed0df02f811932becd8241,2022-12-01-1.ipynb,,Jupyter Notebook,text/plain,2022-12-02T18:45:11+00:00,2022-12-02T18:45:11+00:00,0.0 s,4956,8.0,0.0


In [7]:
%%
```css
th[aria-sort]::after {
    content: attr(aria-sort);
    font-weight: 100;
}
th:empty::before, td:empty::before {
    content: "nan";
}
[data-jp-theme-name="JupyterLab Dark"] section,
[data-jp-theme-name="JupyterLab Dark"] table,
[data-jp-theme-name="JupyterLab Dark"] tr {
    --fg: white; --bg: black;
}
a[href="#demo"] {
    height: 200px;
    background: -moz-element(#gist);
}
```

'th[aria-sort]::after {\n    content: attr(aria-sort);\n    font-weight: 100;\n}\nth:empty::before, td:empty::before {\n    content: "nan";\n}\n[data-jp-theme-name="JupyterLab Dark"] section,\n[data-jp-theme-name="JupyterLab Dark"] table,\n[data-jp-theme-name="JupyterLab Dark"] tr {\n    --fg: white; --bg: black;\n}\na[href="#demo"] {\n    height: 200px;\n    background: -moz-element(#gist);\n}\n'